In [183]:
import solana
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana.transaction import TransactionInstruction, Transaction, AccountMeta
import argparse
from solana.publickey import PublicKey
from solana.blockhash import Blockhash
from solana.system_program import *
import base58, base64
from solana.sysvar import SYSVAR_RENT_PUBKEY
from solana.system_program import SYS_PROGRAM_ID
from spl.token.constants import TOKEN_PROGRAM_ID
from spl.token.client import Token
import time

In [36]:
client = Client("https://api.devnet.solana.com")
ID = "9hmJ5wd2sA6WCutptqJzxPHUe3hbTVNivR5TXRtm1iYC"
#ID = "A4vEBwVMoEZ8j4gthtMnm3MLgKtZoSrw7kYGq4KotE2Q"#"A4vEBwVMoEZ8j4gthtMnm3MLgKtZoSrw7kYGq4KotE2Q"
program_id=base58.b58decode(ID)

In [37]:
k = [239,114,243,183,37,213,148,161,13,172,25,86,143,240,175,157,215,108,156,181,161,230,0,40,228,255,23,209,52,254,120,234,234,209,190,175,172,89,1,135,75,124,43,221,38,148,22,210,210,175,41,10,176,56,96,41,166,103,76,50,58,78,15,74]
God = Keypair().from_secret_key(bytes(k[0:32]))

In [154]:
Alice = Keypair()
Bob = Keypair()

In [ ]:
client.request_airdrop(Alice.public_key, 1000000000)

In [155]:
Alice.public_key

6pabcWVNKzgQ2kW8khLXcQb1FqyG4647AfvdoTVdg2f6

In [242]:
X_mint = Token.create_mint(client, God, God.public_key, 0, TOKEN_PROGRAM_ID)
Y_mint = Token.create_mint(client, God, God.public_key, 0, TOKEN_PROGRAM_ID)

In [243]:
token_program_pk = TOKEN_PROGRAM_ID
x_seed = bytes(X_mint.pubkey)
y_seed = bytes(Y_mint.pubkey)
alice_seed = bytes(Alice.public_key)
bob_seed = bytes(Bob.public_key)
X_vault_pk, _ = PublicKey.find_program_address([x_seed, alice_seed, bob_seed], program_id)
Y_vault_pk, _ = PublicKey.find_program_address([y_seed, alice_seed, bob_seed], program_id)
x_vault_seed = bytes(X_vault_pk)
y_vault_seed = bytes(Y_vault_pk)
escrow_pk, _ = PublicKey.find_program_address([x_vault_seed, y_vault_seed, alice_seed, bob_seed], program_id)

In [244]:
account_metas = [
AccountMeta(Alice.public_key, True, False),
AccountMeta(Bob.public_key, False, False),
AccountMeta(X_mint.pubkey, False, False),
AccountMeta(Y_mint.pubkey, False, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(Y_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(SYS_PROGRAM_ID, False, False),
AccountMeta(SYSVAR_RENT_PUBKEY, False, False),
]

In [245]:
idx = 0
size_A = 10
size_B = 1
instruction_data = idx.to_bytes(1,"little") + size_A.to_bytes(8,"little") + size_B.to_bytes(8,"little")

In [246]:
instruction = TransactionInstruction(data=instruction_data,program_id=ID,keys=account_metas)
tix = Transaction().add(instruction)

In [247]:
#time.sleep(2)
y = client.send_transaction(tix, Alice)

In [248]:
print(y)

{'jsonrpc': '2.0', 'result': '4ogcZ6WKwV6tKRvL1tjrHzb54Jw7SGUR7c5UuCsFYjStSCir4xi3hzjX9coVEtnr5dTFic6U8Gcd9qLwMHi3w1fx', 'id': 316}


In [241]:
print("ESCROW")
print(client.get_account_info(escrow_pk))
print("X VAULT")
print(client.get_account_info(X_vault_pk))

ESCROW
{'jsonrpc': '2.0', 'result': {'context': {'slot': 85820600}, 'value': {'data': ['Vnq4e8SSy/lx8EboVPMVPVQGuDH4Vl6Uv73v4NQQkR0Jhi8GQLfJuXbscorx7xSf078332AQmXPBtHqGiiIcggoAAAAAAAAAAQAAAAAAAACT34O3dWv3/En77iy/1rY2kJhvOoKpxBEMfnqxOWF2dQdt87ixqizlQ0eDVhLysoVXR7+X8SstLV/ra1FlX3wyAP8=', 'base64'], 'executable': False, 'lamports': 1907040, 'owner': '9hmJ5wd2sA6WCutptqJzxPHUe3hbTVNivR5TXRtm1iYC', 'rentEpoch': 198}}, 'id': 298}
X VAULT
{'jsonrpc': '2.0', 'result': {'context': {'slot': 85820600}, 'value': {'data': ['dXlTzsPoQt0TOsrkqNKxvHrnqj2+F0rjvXIiB9kJn0EoV6XuxvjdEO/ZC4Xd4l9VIZSVHxUQ8q1e8a2BHHrPUwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', 'base64'], 'executable': False, 'lamports': 2039280, 'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA', 'rentEpoch': 198}}, 'id': 299}


In [212]:
escrow_pk

7fw3RWhV7gpJHMh5pZVARxSfKLwMfSQCDvxdPhP54bHV